In [5]:
import os
import pandas as pd
from pathlib import Path

In [6]:
Y02_code = "Y02E"
directory = "../data/json_dump"

In [31]:
# Fetch all data from the directory
data = []
for file in os.listdir(directory):
    json_df = pd.read_json(os.path.join(directory, file), lines=True)
    date = file.lstrip("ipg").rstrip(".jsonl")
    date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"
    json_df["upload_date"] = date

    data.append(json_df)
data = pd.concat(data)
# Rename columns
data.columns = ["patent_id", "abstract", "cpc_codes", "uspto_upload_date"]
# Reset index
data = data.reset_index(drop=True)
# Format cpc_codes as concatenated string
data["cpc_codes"] = data["cpc_codes"].apply(", ".join).apply(lambda s: s.replace("\\", ""))
# Set dtype of string columns
data = data.astype("string")
# Re-order columns
data = data[["patent_id", "uspto_upload_date", "cpc_codes", "abstract"]]

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   patent_id          12330 non-null  string
 1   uspto_upload_date  12330 non-null  string
 2   cpc_codes          12330 non-null  string
 3   abstract           12330 non-null  string
dtypes: string(4)
memory usage: 385.4 KB


In [33]:
processed_data_path = Path("../data/processed")
data.to_json(processed_data_path / "uspto_abstracts.json", orient="records")